In [ ]:
#| default_exp datasources.smp
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Serviço Móvel Pessoal
> Módulo para encapsular a extração e processamento do Serviço Móvel Pessoal - Telefonia e Banda Larga Móvel - 2G, 3G, 4G e 5G

In [ ]:
# | export
import os

import pandas as pd
import numpy as np
from dotenv import find_dotenv, load_dotenv

from extracao.constants import (
	AGG_SMP,
	CHANNELS,
	COLUNAS,
	DICT_LICENCIAMENTO,
	MONGO_SMP,
	PROJECTION_LICENCIAMENTO,
)
from extracao.datasources.mosaico import Mosaico
from extracao.location import Geography

In [ ]:
#| export
load_dotenv(find_dotenv())

True

In [ ]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [ ]:
#| export

MONGO_URI = os.environ.get("MONGO_URI")

In [ ]:
#| export
class SMP(Mosaico):
	"""Classe para encapsular a lógica de extração do SMP"""

	def __init__(self, mongo_uri: str = MONGO_URI, limit: int = 0) -> None:
		super().__init__(mongo_uri)
		self.limit = limit

	@property
	def stem(self):
		return 'smp'

	@property
	def collection(self):
		return 'licenciamento'

	@property
	def query(self):
		return MONGO_SMP

	@property
	def projection(self):
		return PROJECTION_LICENCIAMENTO

	@property
	def columns(self):
		return COLUNAS

	@property
	def cols_mapping(self):
		return DICT_LICENCIAMENTO

	def extraction(self) -> pd.DataFrame:
		"""This method returns a DataFrame with the results of the mongo query"""
		pipeline = [{'$match': self.query}, {'$project': self.projection}]
		if self.limit > 0:
			pipeline.append({'$limit': self.limit})
		df = self._extract(self.collection, pipeline)
		df['Log'] = ''
		return df

	def exclude_duplicated(
		self,
		df: pd.DataFrame,  # DataFrame com os dados de Estações
	) -> pd.DataFrame:  # DataFrame com os dados duplicados excluídos
		f"""Exclude the duplicated rows
        Columns considered are defined by the AGG_SMP constant
        """
		df['Estação'] = (
			df['Estação'].astype('string', copy=False).fillna('-1').astype('int', copy=False)
		)
		df = df.sort_values('Estação', ignore_index=True)
		df['Largura_Emissão(kHz)'] = pd.to_numeric(df['Largura_Emissão(kHz)'], errors='coerce')
		# df['Largura_Emissão(kHz)'] = df['Largura_Emissão(kHz)'].fillna(0)
		# df['Classe_Emissão'] = df['Classe_Emissão'].fillna('NI')
		# df['Tecnologia'] = df['Tecnologia'].fillna('NI')
		duplicated = df.duplicated(subset=AGG_SMP, keep='first')
		df_sub = df[~duplicated].copy().reset_index(drop=True)
		# discarded = df[duplicated].copy().reset_index(drop=True)
		# log = f"""[("Colunas", {AGG_SMP}),
		#         ("Processamento", "Registro agrupado e descartado do arquivo final")]"""
		# self.append2discarded(self.register_log(discarded, log))
		# for col in AGG_SMP:
		#     discarded_with_na = df_sub[df_sub[col].isna()]
		#     log = f"""[("Colunas", {col}),
		#             ("Processamento", "Registro com valor nulo presente")]"""
		#     self.append2discarded(self.register_log(discarded_with_na, log))
		df_sub.dropna(subset=AGG_SMP, inplace=True)
		df_sub['Multiplicidade'] = (
			df.groupby(AGG_SMP, dropna=True, sort=False, observed=True).size().values
		)
		log = f'[("Colunas", {AGG_SMP}), ("Processamento", "Agrupamento")]'
		return self.register_log(df_sub, log, df_sub['Multiplicidade'] > 1)

	@staticmethod
	def read_channels():
		"""Reads and formats the SMP channels files"""
		channels = pd.read_csv(CHANNELS, dtype='string')
		cols = ['Downlink_Inicial', 'Downlink_Final', 'Uplink_Inicial', 'Uplink_Final']
		channels[cols] = channels[cols].astype('float')
		channels = channels.sort_values(['Downlink_Inicial'], ignore_index=True)
		channels['N_Bloco'] = channels['N_Bloco'].str.strip()
		channels['Faixa'] = channels['Faixa'].str.strip()
		return channels

	def exclude_invalid_channels(
		self,
		df: pd.DataFrame,  # DataFrame de Origem
	) -> pd.DataFrame:  # DataFrame com os canais inválidos excluídos
		"""Helper function to keep only the valid downlink channels"""
		df_sub = df[df.Canalização == 'Downlink'].reset_index(drop=True)
		# for flag in ["Uplink", "Inválida"]:
		#     discarded = df[df.Canalização == flag]
		#     if not discarded.empty:
		#         log = f"""[("Colunas", ("Frequência", "Largura_Emissão(kHz)")),
		#                  ("Processamento", "Canalização {flag}")]"""
		#         self.append2discarded(self.register_log(discarded, log))
		return df_sub

	def validate_channels(
		self,
		df: pd.DataFrame,  # DataFrame with the original channels info
	) -> pd.DataFrame:  # DataFrame with the channels validated and added info
		"""Read the SMP channels file, validate and merge the channels present in df"""
		bw = df['Largura_Emissão(kHz)'].astype('float') / 2000  # Unidade em kHz
		df['Início_Canal_Down'] = df.Frequência.astype(float) - bw
		df['Fim_Canal_Down'] = df.Frequência.astype(float) + bw
		channels = self.read_channels()
		grouped_channels = df.groupby(
			['Início_Canal_Down', 'Fim_Canal_Down'], as_index=False
		).size()
		grouped_channels.sort_values('size', ascending=False, inplace=True, ignore_index=True)
		grouped_channels['Canalização'] = 'Inválida'
		grouped_channels['Offset'] = np.nan
		grouped_channels['Blocos_Downlink'] = pd.NA
		grouped_channels['Faixas'] = pd.NA
		grouped_channels[['Blocos_Downlink', 'Faixas', 'Canalização']] = grouped_channels[
			['Blocos_Downlink', 'Faixas', 'Canalização']
		].astype('string', copy=False)
		grouped_channels['Offset'] = grouped_channels['Offset'].astype('float', copy=False)

		for row in grouped_channels.itertuples():
			interval = channels[
				(row.Início_Canal_Down < channels['Downlink_Final'])
				& (row.Fim_Canal_Down > channels['Downlink_Inicial'])
			]
			faixa = 'Downlink'
			if interval.empty:
				interval = channels[
					(row.Início_Canal_Down < channels['Uplink_Final'])
					& (row.Fim_Canal_Down > channels['Uplink_Inicial'])
				]
				if interval.empty:
					continue
				faixa = 'Uplink'

			down = ' | '.join(
				interval[['Downlink_Inicial', 'Downlink_Final']].apply(
					lambda x: f'{x.iloc[0]}-{x.iloc[1]}', axis=1
				)
			)
			faixas = ' | '.join(interval.Faixa.unique())
			if len(offset := interval.Offset.unique()) != 1:
				continue
			grouped_channels.loc[
				row.Index, ['Blocos_Downlink', 'Faixas', 'Canalização', 'Offset']
			] = (down, faixas, faixa, float(offset[0]))
		grouped_channels = grouped_channels[
			[
				'Início_Canal_Down',
				'Fim_Canal_Down',
				'Blocos_Downlink',
				'Faixas',
				'Canalização',
				'Offset',
			]
		]
		df = pd.merge(df, grouped_channels, how='left', on=['Início_Canal_Down', 'Fim_Canal_Down'])
		return self.exclude_invalid_channels(df)

	def generate_uplink(
		self,
		df: pd.DataFrame,  # Source dataFrame with downlink frequencies and offset
	) -> pd.DataFrame:  # DataFrame with the uplink frequencies added
		"""Generate the respective Uplink channels based on the Downlink frequencies and Offset"""
		df['Offset'] = pd.to_numeric(df['Offset'], errors='coerce').astype('float')
		df['Largura_Emissão(kHz)'] = pd.to_numeric(
			df['Largura_Emissão(kHz)'], errors='coerce'
		).astype('float')
		valid = (
			(df.Offset.notna())
			& (~np.isclose(df.Offset, 0))
			& (df['Largura_Emissão(kHz)'].notna())
			& (~np.isclose(df['Largura_Emissão(kHz)'], 0))
		)
		df[['Frequência', 'Offset']] = df[['Frequência', 'Offset']].astype('float')
		df.loc[valid, 'Frequência_Recepção'] = df.loc[valid, 'Frequência'] - df.loc[valid, 'Offset']
		return df

	def substitute_coordinates(
		self,
		df: pd.DataFrame,  # Source dataframe
	) -> pd.DataFrame:  # Source dataframe with coordinates replace for the city one
		"""Substitute the coordinates for the central coordinates of the municipality
		Only does it for the grouped rows (Multiplicity > 1) since for these rows the
		coordinate values are no longer valid.

		"""
		geo = Geography(df)
		df = geo.merge_df_with_ibge(df)
		rows = df.Multiplicidade > 1
		df.loc[rows, 'Latitude'] = df.loc[rows, 'Latitude_IBGE'].copy()
		df.loc[rows, 'Longitude'] = df.loc[rows, 'Longitude_IBGE'].copy()
		log = """[("Colunas", ("Latitude", "Longitude")), 
        ("Processamento", "Substituição por Coordenadas do Município (Agrupamento)")]"""
		return self.register_log(df, log, df.Multiplicidade > 1)

	def input_fixed_columns(
		self,
		df: pd.DataFrame,  # Source dataframe
	) -> pd.DataFrame:  # Cleaned dataframe with some additional columns added
		"""Formats and adds some helper columns to the dataframe"""
		df['Status'] = 'L'
		df['Serviço'] = '010'
		down = df.drop('Frequência_Recepção', axis=1)
		down['Fonte'] = 'MOSAICO-LICENCIAMENTO'
		down['Classe'] = 'FB'
		up = df.drop('Frequência', axis=1)
		up = up.rename(columns={'Frequência_Recepção': 'Frequência'})
		up.dropna(subset='Frequência', inplace=True)
		up['Fonte'] = 'CANALIZACAO-SMP'
		up['Classe'] = 'ML'
		return pd.concat([down, up], ignore_index=True)

	def _format(
		self,
		df: pd.DataFrame,  # Source dataframe
	) -> pd.DataFrame:  # Final processed dataframe
		"""Formats, cleans, groups, adds and standardizes the queried data from the database"""
		df = df.rename(columns=self.cols_mapping)
		df = self.split_designacao(df)
		df = self.exclude_duplicated(df)
		df = self.validate_channels(df)
		df = self.generate_uplink(df)
		df = self.substitute_coordenates(df)
		df = self.input_fixed_columns(df)
		return df.loc[:, self.columns]

In [ ]:
#|eval:false
smp = SMP(limit=100000)

In [ ]:
# |eval:false
e = smp.extraction()

In [ ]:
e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   _id                   100000 non-null  string
 1   NumAto                100000 non-null  string
 2   DataValidade          100000 non-null  string
 3   NumFistel             100000 non-null  string
 4   NomeEntidade          100000 non-null  string
 5   NumEstacao            100000 non-null  string
 6   Latitude              100000 non-null  string
 7   Longitude             100000 non-null  string
 8   CodTipoClasseEstacao  100000 non-null  string
 9   CodMunicipio          100000 non-null  string
 10  SiglaUf               100000 non-null  string
 11  DesignacaoEmissao     100000 non-null  string
 12  NumServico            100000 non-null  string
 13  formId                100000 non-null  string
 14  Tecnologia            100000 non-null  string
 15  FreqTxMHz         

In [ ]:
client = smp.connect()

In [ ]:
client

MongoClient(host=['anatelbdro06:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000, connecttimeoutms=10000, authsource='admin')

In [ ]:
database = client[smp.database]

In [ ]:
database

Database(MongoClient(host=['anatelbdro06:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000, connecttimeoutms=10000, authsource='admin'), 'sms')

In [ ]:
collection = database[smp.collection]
collection

Collection(Database(MongoClient(host=['anatelbdro06:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000, connecttimeoutms=10000, authsource='admin'), 'sms'), 'licenciamento')

In [ ]:
collection.find_one()

{'_id': '4d5c019f58992',
 'NumAto': 73832008.0,
 'DataValidade': '2023-12-15',
 'NumFistel': '50409428698',
 'NumCnpjCpf': '02421421000111',
 'NomeEntidade': 'TIM S/A',
 'IdtEstacao': '3058301',
 'NumEstacao': '1058',
 'NomeIndicativo': 'ALFXS_0001',
 'Modalidade': 'ERB com 1 ou mais Equip/Ant. no Setor',
 'Latitude': -9.24111,
 'Longitude': -35.77944,
 'ObservacaoLicenca': '',
 'CodMotivoExclusao': 'EE',
 'DataExclusao': '',
 'CodTipoClasseEstacao': 'FB',
 'FreqInicialMHz': '869.000',
 'FreqFinalMHz': '880.000',
 'FreqCentralMHz': '874.5',
 'CodCep': '57995000',
 'EnderecoEstacao': 'RODOVIA BR 101 NORTE 0 MORRO CORTE NOVO',
 'CodMunicipio': '2702801',
 'SiglaUf': 'AL',
 'CodEquipamentoTransmissor': '032341303903',
 'NumSetor': '7',
 'CodProdutoTransmissor': '55065',
 'PotenciaTransmissorWatts': '80.000',
 'IdtUnidadePotenciaOperacao': '2',
 'DesignacaoEmissao': '5M00G7W',
 'CodEquipamentoAntena': '012720905344',
 'CodProdutoAntena': '28359',
 'CodTipoAntena': '396',
 'Polarizacao': 'X

In [ ]:
pipeline = [{'$match': smp.query}, {'$project': smp.projection}, {'$limit': 10000}]
pipeline

[{'$match': {'$and': [{'DataExclusao': None},
    {'DataValidade': {'$nin': ['', None]}},
    {'Status.state': 'LIC-LIC-01'},
    {'NumServico': '010'},
    {'FreqTxMHz': {'$nin': [None, '', 0], '$type': 1.0}},
    {'CodMunicipio': {'$nin': [None, '']}},
    {'NumFistel': {'$nin': [None, '']}},
    {'CodTipoClasseEstacao': {'$nin': [None, '']}},
    {'DesignacaoEmissao': {'$nin': [None, '']}},
    {'Tecnologia': {'$nin': [None, '']}}]}},
 {'$project': {'NumAto': 1.0,
   'NumFistel': 1.0,
   'NumServico': 1.0,
   'NomeEntidade': 1.0,
   'SiglaUf': 1.0,
   'NumEstacao': 1.0,
   'CodTipoClasseEstacao': 1.0,
   'NomeMunicipio': 1.0,
   'CodMunicipio': 1.0,
   'DataValidade': 1.0,
   'FreqTxMHz': 1.0,
   'formId': 1.0,
   'Tecnologia': 1.0,
   'Latitude': 1.0,
   'Longitude': 1.0,
   'DesignacaoEmissao': 1.0}},
 {'$limit': 10000}]

In [ ]:
result = collection.aggregate(pipeline)

In [ ]:
result.next()

{'_id': '4d5c019f589c0',
 'NumAto': 48412023,
 'DataValidade': '2038-04-30',
 'NumFistel': '50409146366',
 'NomeEntidade': 'TELEFONICA BRASIL S.A.',
 'NumEstacao': '3549',
 'Latitude': -28.50528,
 'Longitude': -50.93556,
 'CodTipoClasseEstacao': 'FB',
 'CodMunicipio': '4322509',
 'SiglaUf': 'RS',
 'DesignacaoEmissao': '5M00D7W',
 'NumServico': '010',
 'formId': 'base',
 'Tecnologia': 'WCDMA',
 'FreqTxMHz': 2160.0,
 'NomeMunicipio': 'VACARIA'}

In [ ]:
result = list(result)

In [ ]:
result

[{'_id': '4d5c019f589c1',
  'NumAto': 48412023,
  'DataValidade': '2038-04-30',
  'NumFistel': '50409146366',
  'NomeEntidade': 'TELEFONICA BRASIL S.A.',
  'NumEstacao': '3549',
  'Latitude': -28.50528,
  'Longitude': -50.93556,
  'CodTipoClasseEstacao': 'FB',
  'CodMunicipio': '4322509',
  'SiglaUf': 'RS',
  'DesignacaoEmissao': '5M00D7W',
  'NumServico': '010',
  'formId': 'base',
  'Tecnologia': 'WCDMA',
  'FreqTxMHz': 2160.0,
  'NomeMunicipio': 'VACARIA'},
 {'_id': '4d5c019f589c2',
  'NumAto': 48412023,
  'DataValidade': '2038-04-30',
  'NumFistel': '50409146366',
  'NomeEntidade': 'TELEFONICA BRASIL S.A.',
  'NumEstacao': '3549',
  'Latitude': -28.50528,
  'Longitude': -50.93556,
  'CodTipoClasseEstacao': 'FB',
  'CodMunicipio': '4322509',
  'SiglaUf': 'RS',
  'DesignacaoEmissao': '5M00G9W',
  'NumServico': '010',
  'formId': 'base',
  'Tecnologia': 'WCDMA',
  'FreqTxMHz': 2167.5,
  'NomeMunicipio': 'VACARIA'},
 {'_id': '4d5c019f589cf',
  'NumAto': 48412023,
  'DataValidade': '203

In [ ]:
df = pd.DataFrame(result, copy=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   9999 non-null   object 
 1   NumAto                9999 non-null   float64
 2   DataValidade          9999 non-null   object 
 3   NumFistel             9999 non-null   object 
 4   NomeEntidade          9999 non-null   object 
 5   NumEstacao            9999 non-null   object 
 6   Latitude              9999 non-null   object 
 7   Longitude             9999 non-null   object 
 8   CodTipoClasseEstacao  9999 non-null   object 
 9   CodMunicipio          9999 non-null   object 
 10  SiglaUf               9999 non-null   object 
 11  DesignacaoEmissao     9999 non-null   object 
 12  NumServico            9999 non-null   object 
 13  formId                9999 non-null   object 
 14  Tecnologia            9999 non-null   object 
 15  FreqTxMHz            

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   0 non-null      object 
 1   NumAto                9999 non-null   float64
 2   DataValidade          0 non-null      object 
 3   NumFistel             0 non-null      object 
 4   NomeEntidade          0 non-null      object 
 5   NumEstacao            0 non-null      object 
 6   Latitude              9936 non-null   object 
 7   Longitude             9936 non-null   object 
 8   CodTipoClasseEstacao  0 non-null      object 
 9   CodMunicipio          0 non-null      object 
 10  SiglaUf               0 non-null      object 
 11  DesignacaoEmissao     0 non-null      object 
 12  NumServico            0 non-null      object 
 13  formId                0 non-null      object 
 14  Tecnologia            0 non-null      object 
 15  FreqTxMHz            

In [ ]:
#|eval:false